In [4]:
!pip install beautifulsoup4 datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [5]:
# Importing necessary Libraries
import torch
import pandas as pd
import numpy as np
import spacy as spacy
from bs4 import BeautifulSoup
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # Tokenizer for bert-base-uncase model. Evry model cames with its tokenizer
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")  # Loading the model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
imdb_movie_reviews = load_dataset('imdb') # Loading the dataset
# Visualizing the datasets
train_data = pd.DataFrame(imdb_movie_reviews['train'])
test_data = pd.DataFrame(imdb_movie_reviews['test'])

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
train_data.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


In [9]:
nlp = spacy.load("en_core_web_sm") # Loading the spacy model
def preprocess_text(text):
  # Remove HTML Tags
  text = BeautifulSoup(text, "html.parser").get_text()
  doc = nlp(text.lower())
  tokens = [token.lemma_ for token in doc]
  return " ".join(tokens)

sample_text = train_data['text'][0]
print(sample_text)
print(preprocess_text(sample_text))

I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, eve

In [11]:
df_0 = train_data[train_data['label'] == 0]
df_1 = train_data[train_data['label'] == 1]

df_0_sample = df_0.sample(7000)
df_1_sample = df_1.sample(2000)

In [12]:
df = pd.concat([df_0_sample, df_1_sample])

In [17]:
df_0_test = test_data[test_data['label'] == 0]
df_1_test = test_data[test_data['label'] == 1]

df_0_sample_t = df_0_test.sample(1000)
df_1_sample_t = df_1_test.sample(500)
df_test = pd.concat([df_0_sample_t, df_1_sample_t])

In [14]:
df["label"].value_counts()

,count
label,
0,7000
1,2000


In [15]:
# Cleaning the entire data
df["cleaned_review"] = df["text"].apply(preprocess_text)

<ipython-input-9-f53549d27a15>:4: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [18]:
df_test["cleaned_review"] = df_test["text"].apply(preprocess_text)

In [20]:
df_test.drop(columns=["text"], inplace=True)
df.drop(columns=["text"], inplace=True)

In [21]:
df.head()

,label,cleaned_review
2224,0,be this movie stupid ? yup . do this movie dep...
8771,0,this movie be absolute torture . first of all ...
7507,0,probably the only thing that get the movie up ...
11076,0,"I be a big fan of the true crime genre , but I..."
7890,0,"no plot , crappy acting , and pointless gore ...."


In [22]:
train_encodings = tokenizer(list(df["cleaned_review"]), truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(list(df_test["cleaned_review"]), truncation=True, padding=True, max_length=128)

In [23]:
train_labels = df["label"].values
test_labels = df_test["label"].values

In [25]:
train_encodings.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [42]:
class SentimentDatasets(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    item = {key:  torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item["labels"] = torch.tensor(self.labels[idx])
    return item
  def __len__(self):
    return len(self.labels)

In [43]:

# Create datasets for train and test
train_dataset = SentimentDatasets(train_encodings, train_labels)
test_dataset = SentimentDatasets(test_encodings, test_labels)

In [44]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

# Load pre-trained BERT model with a classification head
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # Adjust num_labels as needed

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
      warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

In [46]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
)
trainer.train()

Step,Training Loss
10,0.579000
20,0.558700
30,0.539000
40,0.535700
50,0.501500
60,0.445500
70,0.560600
80,0.564400
90,0.626800
100,0.510600


TrainOutput(global_step=3375, training_loss=0.24623005042059554, metrics={'train_runtime': 780.1891, 'train_samples_per_second': 34.607, 'train_steps_per_second': 4.326, 'total_flos': 1775999623680000.0, 'train_loss': 0.24623005042059554, 'epoch': 3.0})

In [47]:
results = trainer.evaluate()

In [48]:
print(results)

{'eval_loss': 0.6663809418678284, 'eval_runtime': 10.6621, 'eval_samples_per_second': 140.685, 'eval_steps_per_second': 17.633, 'epoch': 3.0}


In [51]:
def MakeInference(text):
  inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
  inputs = inputs.to(model.device)
  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
  return predicted_class

In [75]:
df.iloc[7410]["cleaned_review"]

'cat soup be a short anime base on the legendary manga nekojiru . it win the award " good short film " at the 6th fantasia film festival and also win the " excellence prize " at japan \'s medium art festival.when little kitten nyaako \'s soul be steal by death , she and her brother nyatta embark on a bizarre journey to get it back . in the surreal dreamscape of the other side , they encounter many fantastic character and remarkable , often disturb adventures.cat soup be an anime like nothing you have ever see . it be hello kitty on acid ! it be very original , stunningly beautiful and possess a great sense of strangeness and lyricism . cat soup be very surrealistic ( there be no dialogue ) and sometimes cruel and gory . so it be more an anime for adult than child ( they may not understand at all ! ) . a great journey for those who get the chance to see this absolute masterpiece . an must - see !'

In [78]:
sample_text = "This movie is not good at all please dont try to watch it"
predicted = MakeInference(sample_text)
print(predicted)

0


In [79]:
df_test.shape

(1500, 2)

In [93]:
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
test_data = DataLoader(test_dataset, batch_size=1)
def calculate_accuracy():
  aall_predictions = []
  all_labels = []
  with torch.no_grad():
    for batch in test_data:
      inputs = {k: v.to(model.device) for k, v in batch.items() if k != "labels"}
      labels = batch["labels"].to(model.device)
      outputs = model(**inputs)
      logits = outputs.logits
      predicted_class = torch.argmax(logits, dim=1) # Remove .item() to keep it as a tensor
      aall_predictions.extend(predicted_class.cpu().numpy())  # Move predictions to CPU and add to list
      all_labels.extend(labels.cpu().numpy())  # Move labels to CPU and add to list
  accuracy = accuracy_score(all_labels, aall_predictions)
  return accuracy, aall_predictions, all_labels

In [94]:
acc, y_pred, y = calculate_accuracy()

In [95]:
from sklearn.metrics import classification_report
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.96      0.91      1000
           1       0.89      0.71      0.79       500

    accuracy                           0.87      1500
   macro avg       0.88      0.83      0.85      1500
weighted avg       0.88      0.87      0.87      1500

